# Bought By Many Interview Exercise

In [1]:
import os
import json
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

## 1. Process input files into CSVs

First, let's take a look at some policies to see what we're dealing with.

In [2]:
with open(f'data/policies/{os.listdir("data/policies")[0]}') as file:
    text = file.read()
    policy = json.loads(text)

print(json.dumps(policy, indent=2))

{
  "_id": "5d3dbfd80f17bd015b94cce4",
  "created_at": 1564327896418,
  "updated_at": 1578003371674,
  "account_ref": "BBM",
  "affiliate_code": "MSM",
  "cessation_date": 1595894400000,
  "data": {
    "insured_entities": [
      {
        "breed": "PAPILLON",
        "breed_friendly": "Papillon",
        "dob": 1546214400000,
        "gender": "MALE",
        "healthy": true,
        "microchipped": true,
        "pedigree": true,
        "pedigree_type": "PEDIGREE",
        "pet_address_postcode": "NN3",
        "pet_name": "George",
        "photo": "",
        "ref": "341692",
        "spayed_neutered": false,
        "species": "DOG",
        "uuid": "dcee9cc0-7987-40cb-a428-5bde29a1f057",
        "value": 1000
      }
    ],
    "payment": {
      "direct_debit_account_details": "gAAAAABdPb_ElU0tFUu144uNYZkLMiYwncHrQPK9ixRKauSkkGLR7TGgsqUg0MJ5YCOqxj03WlAXF-NiO3i5Nrt7aPRUI7afOY_cdyi1rFAS66IdO_PWhhE=",
      "direct_debit_account_name": "<removed>",
      "method": "IBSUITE_DIRECT

We see a lot of information, including details of:

* the policy holder
* the insured pets
* the insurance products included in the policy (allowances, pricing, rules etc.)
* payments
* event timelines (on risk, started, cancelled etc.)